In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

# Define the Graph Encoder
class GraphEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphEncoder, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Define the Modality Encoder (e.g., for image or text)
class ModalityEncoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ModalityEncoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Contrastive Loss Function
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z1, z2):
        batch_size = z1.size(0)
        z1 = F.normalize(z1, dim=1)
        z2 = F.normalize(z2, dim=1)

        # Similarity matrix
        sim_matrix = torch.exp(torch.mm(z1, z2.t()) / self.temperature)
        pos_sim = torch.exp(torch.sum(z1 * z2, dim=-1) / self.temperature)

        loss = -torch.log(pos_sim / (sim_matrix.sum(dim=-1) + 1e-8)).mean()
        return loss

# Multimodal Graph Contrastive Model
class MultimodalGraphCL(nn.Module):
    def __init__(self, graph_input_dim, modality_input_dim, hidden_dim, output_dim):
        super(MultimodalGraphCL, self).__init__()
        self.graph_encoder = GraphEncoder(graph_input_dim, hidden_dim, output_dim)
        self.modality_encoder = ModalityEncoder(modality_input_dim, output_dim)
        self.contrastive_loss = ContrastiveLoss()

    def forward(self, graph_data, modality_data):
        # Graph encoding
        z1 = self.graph_encoder(graph_data.x, graph_data.edge_index)

        # Modality encoding
        z2 = self.modality_encoder(modality_data)

        # Contrastive loss
        loss = self.contrastive_loss(z1, z2)
        return loss

# Dummy data for demonstration
num_nodes = 10
graph_input_dim = 5
modality_input_dim = 100
hidden_dim = 64
output_dim = 32

# Create dummy graph data
x = torch.rand((num_nodes, graph_input_dim)) # Node features
edge_index = torch.randint(0, num_nodes, (2, 20)) # Random edges
graph_data = Data(x=x, edge_index=edge_index)

# Dummy modality data (e.g., 10 samples with modality_input_dim features)
modality_data = torch.rand((num_nodes, modality_input_dim))

# Initialize and train the model
model = MultimodalGraphCL(graph_input_dim, modality_input_dim, hidden_dim, output_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training step
model.train()
for epoch in range(100):
    optimizer.zero_grad()
    loss = model(graph_data, modality_data)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: 2.3242
Epoch 2, Loss: 2.2935
Epoch 3, Loss: 2.2677
Epoch 4, Loss: 2.2422
Epoch 5, Loss: 2.2129
Epoch 6, Loss: 2.1772
Epoch 7, Loss: 2.1334
Epoch 8, Loss: 2.0800
Epoch 9, Loss: 2.0178
Epoch 10, Loss: 1.9521
Epoch 11, Loss: 1.8897
Epoch 12, Loss: 1.8338
Epoch 13, Loss: 1.7821
Epoch 14, Loss: 1.7330
Epoch 15, Loss: 1.6863
Epoch 16, Loss: 1.6390
Epoch 17, Loss: 1.5894
Epoch 18, Loss: 1.5390
Epoch 19, Loss: 1.4884
Epoch 20, Loss: 1.4370
Epoch 21, Loss: 1.3854
Epoch 22, Loss: 1.3352
Epoch 23, Loss: 1.2886
Epoch 24, Loss: 1.2479
Epoch 25, Loss: 1.2121
Epoch 26, Loss: 1.1798
Epoch 27, Loss: 1.1494
Epoch 28, Loss: 1.1203
Epoch 29, Loss: 1.0928
Epoch 30, Loss: 1.0703
Epoch 31, Loss: 1.0576
Epoch 32, Loss: 1.0543
Epoch 33, Loss: 1.0548
Epoch 34, Loss: 1.0532
Epoch 35, Loss: 1.0468
Epoch 36, Loss: 1.0365
Epoch 37, Loss: 1.0241
Epoch 38, Loss: 1.0120
Epoch 39, Loss: 1.0018
Epoch 40, Loss: 0.9946
Epoch 41, Loss: 0.9903
Epoch 42, Loss: 0.9886
Epoch 43, Loss: 0.9882
Epoch 44, Loss: 0.98